In [1]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint
# import config
import re
from sqlalchemy import create_engine

In [2]:
# Import json and convert to DF
url = 'https://www.thecocktaildb.com/api/json/v2/9973533/search.php?s'

response = requests.get(url).json()

drinks_df = pd.json_normalize(response['drinks'], max_level=1)
drinks_df.head()

,idDrink,strDrink,strDrinkAlternate,strTags,strVideo,strCategory,strIBA,strAlcoholic,strGlass,strInstructions,...,strMeasure10,strMeasure11,strMeasure12,strMeasure13,strMeasure14,strMeasure15,strImageSource,strImageAttribution,strCreativeCommonsConfirmed,dateModified
0,15997,GG,None,None,None,Ordinary Drink,None,Optional alcohol,Collins Glass,Pour the Galliano liqueur over ice. Fill the r...,...,None,None,None,None,None,None,None,None,No,2016-07-18 22:06:00
1,17222,A1,None,None,None,Cocktail,None,Alcoholic,Cocktail glass,"Pour all ingredients into a cocktail shaker, m...",...,None,None,None,None,None,None,None,None,No,2017-09-07 21:42:09
2,13501,ABC,None,None,None,Shot,None,Alcoholic,Shot glass,Layered in a shot glass.,...,None,None,None,None,None,None,None,None,No,2016-08-31 19:32:08
3,17203,Kir,None,"IBA,ContemporaryClassic",None,Ordinary Drink,None,Alcoholic,Wine Glass,Add the crème de cassis to the bottom of the g...,...,None,None,None,None,None,None,None,None,No,2017-09-02 17:38:14
4,14229,747,None,None,None,Shot,None,Alcoholic,Shot glass,"pour kaluha, then Baileys, then Frangelico not...",...,None,None,None,None,None,None,None,None,No,2016-08-31 19:28:26


In [3]:
measure_df = drinks_df[['idDrink', 'strDrink',  'strIngredient1', 'strImageSource', 'strMeasure1', 'strIngredient2', 'strMeasure2',
                        'strIngredient3', 'strMeasure3', 'strIngredient4', 'strMeasure4','strIngredient5', 'strMeasure5',
                        'strIngredient6', 'strMeasure6', 'strIngredient7', 'strMeasure7','strIngredient8', 'strMeasure8',
                        'strIngredient9', 'strMeasure9', 'strIngredient10', 'strMeasure10', 'strIngredient11', 'strMeasure11',
                        'strIngredient12', 'strMeasure12']]

measure_df

,idDrink,strDrink,strIngredient1,strImageSource,strMeasure1,strIngredient2,strMeasure2,strIngredient3,strMeasure3,strIngredient4,...,strIngredient8,strMeasure8,strIngredient9,strMeasure9,strIngredient10,strMeasure10,strIngredient11,strMeasure11,strIngredient12,strMeasure12
0,15997,GG,Galliano,None,2 1/2 shots,Ginger ale,None,Ice,None,None,...,None,None,None,None,None,None,None,None,None,None
1,17222,A1,Gin,None,1 3/4 shot,Grand Marnier,1 Shot,Lemon Juice,1/4 Shot,Grenadine,...,None,None,None,None,None,None,None,None,None,None
2,13501,ABC,Amaretto,None,1/3,Baileys irish cream,1/3,Cognac,1/3,None,...,None,None,None,None,None,None,None,None,None,None
3,17203,Kir,Creme de Cassis,None,1 part,Champagne,5 parts,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,14229,747,Kahlua,None,1/3 part,Baileys irish cream,1/3 part,Frangelico,1/3 part,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,16100,Amaretto Stone Sour Alternative,Sour mix,None,2 oz,Amaretto,2 oz,Tequila,2 oz,Orange juice,...,None,None,None,None,None,None,None,None,None,None
614,12658,Banana Strawberry Shake Daiquiri,Strawberries,None,1/2 lb frozen,Banana,1 frozen,Apple juice,2 cups fresh,None,...,None,None,None,None,None,None,None,None,None,None
615,16984,Radioactive Long Island Iced Tea,Rum,None,1 oz,Vodka,1 oz,Tequila,1 oz,Gin,...,Malibu rum,1 oz,None,None,None,None,None,None,None,None
616,14029,57 Chevy with a White License Plate,Creme de Cacao,None,1 oz white,Vodka,1 oz,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [4]:
measurement_conversions = {'Units': ['','cL','qt','shot','spoons','Dash','Cup','cans','quart','drop','L','Dashes','jigger','bottle',
                'fifth','top','part','Fill','shots','beaten','tablespoons','inch','gr','tbsp','handful','pinches',
                'kg','tsp','cubes','cup','tblsp','splash','dl','can','pint','parts','full','Shot','Full','Bacardi',
                'jiggers','drops','Pinch','cube','oz','splashes','cl','Float','lb','ml','bottles','scoops','Squeeze',
                'fill','package','packages','dash','dashes','gal','seltzer','Juice','cups','Top'],
                             'multiplier': [0,0.338,32,1.5,0.5,0.25,8,12,32,0.01,33.8,0.25,1.5,12,
                                           25.5,1,1.5,1,1.5,2,0.5,1,0.035,0.5,1,0.15,
                                           35.274,0.167,0.5,8,0.5,0.5,3.38,12,16,1.5,1,1.5,1,1.5,
                                           1.5,0.01,0.15,0.5,1,0.5,0.338,1,16,0.0338,12,2,0.25,
                                           1,1,1,0.25,0.25,124,1,1,8,1]}
measurement_conversions_df = pd.DataFrame.from_dict(measurement_conversions)

measurement_conversions_df.head(10)

,Units,multiplier
0,,0.000
1,cL,0.338
2,qt,32.000
3,shot,1.500
4,spoons,0.500
5,Dash,0.250
6,Cup,8.000
7,cans,12.000
8,quart,32.000
9,drop,0.010


In [5]:
def units_to_oz(row):
    unit = row["Units"]
    #if unit in measurement_conversions_df, then
    if len(measurement_conversions_df.loc[measurement_conversions_df["Units"] == unit]) > 0:
        mult = measurement_conversions_df.loc[measurement_conversions_df["Units"] == unit, "multiplier"].item()
        t = row["Total"]
        row["Total"] = t * mult
        if row["Total"] == None:
            row["Total"] = 0
    else:
        row["Total"] = 0

    return row

In [6]:
count = 0

def split(df_column):
    global count
#     print("count:", count)
    count += 1
#     print(type(df_column))
#     df_column = str(df_column)

    ### this section separates numbers from words
    measurements = []
    units = []
    
    try: 
        for index, string_list in df_column.items():
            #string_list = row['strMeasure1']
            if (string_list):
                    wordsRegex = "[a-zA-z]+"
                    numbersRegex = "(\d [\d\/\d]*)"
                    words = re.search(wordsRegex, string_list)
                    measure = re.search(numbersRegex, string_list)
                    if (words):
                        units.append(words.group(0))
                    else:
                        units.append("")
                    if (measure):
                        measurements.append(measure.group(0))
                    else:
                        measurements.append("")
            else:
                measurements.append("")
                units.append("")

        measurements_df = pd.DataFrame(
            {"Measurements": measurements, "Units": units}, 
        )
        #print(measurements_df)

        ### this section converts the fractions to floats

        # split measurements column on the space, rename columns 
        split_measurements_df = pd.DataFrame(measurements_df['Measurements'].str.split(' ',1).tolist(),
                                         columns = ['Measurement_a','Measurement_b'])

        # Potential fractions to be dealt with: ['1/2', '3/4', '2/3', '1/4']
        # replace with decimals
        split_measurements_df['Measurement_b'] = split_measurements_df['Measurement_b'].replace(['1/2','3/4','2/3','1/4'],
                                                                                                [.5,.75,.66,.25])
        # convert string to number
        split_measurements_df['Measurement_a'] = pd.to_numeric(split_measurements_df['Measurement_a'], errors='coerce')
        split_measurements_df['Measurement_b'] = pd.to_numeric(split_measurements_df['Measurement_b'], errors='coerce')

        # make all NANs = 0
        split_measurements_df['Measurement_b'] = split_measurements_df['Measurement_b'].fillna(0)

        # create a new column with total
        split_measurements_df['Total'] = split_measurements_df['Measurement_a'] + split_measurements_df['Measurement_b']

        # add back the units column
        split_measurements_df['Units'] = units
        #split_measurements_df

        ### this section converts to ounces using pre-defined multipliers
        # use units_to_oz with apply function
        normalized_measurements_df = split_measurements_df.apply(units_to_oz, axis = 1)

        df_column = normalized_measurements_df["Total"]
    
    except:
        pass
    
    return df_column # this returns a series

In [7]:
new_df = measure_df[['strMeasure1', 'strMeasure2',
       'strMeasure3', 'strMeasure4', 'strMeasure5', 'strMeasure6',
       'strMeasure7', 'strMeasure8', 'strMeasure9', 'strMeasure10',
       'strMeasure11', 'strMeasure12']].apply(split)
clean_measurements = new_df.fillna(0)
clean_measurements

,strMeasure1,strMeasure2,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,strMeasure11,strMeasure12
0,3.750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.625,1.5,6.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.500,7.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.500,4.5,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
613,2.000,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
614,32.000,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
615,1.000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
616,1.000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
clean_ingredients = measure_df[['idDrink', 'strDrink', 'strImageSource', 'strIngredient1', 'strIngredient2',
                        'strIngredient3', 'strIngredient4','strIngredient5',
                        'strIngredient6','strIngredient7','strIngredient8',
                        'strIngredient9','strIngredient10','strIngredient11',
                        'strIngredient12']]
result = pd.concat([clean_ingredients, clean_measurements], axis=1)
result

,idDrink,strDrink,strImageSource,strIngredient1,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,...,strMeasure3,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,strMeasure11,strMeasure12
0,15997,GG,None,Galliano,Ginger ale,Ice,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,17222,A1,None,Gin,Grand Marnier,Lemon Juice,Grenadine,None,None,None,...,6.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,13501,ABC,None,Amaretto,Baileys irish cream,Cognac,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,17203,Kir,None,Creme de Cassis,Champagne,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,14229,747,None,Kahlua,Baileys irish cream,Frangelico,None,None,None,None,...,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,16100,Amaretto Stone Sour Alternative,None,Sour mix,Amaretto,Tequila,Orange juice,None,None,None,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
614,12658,Banana Strawberry Shake Daiquiri,None,Strawberries,Banana,Apple juice,None,None,None,None,...,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
615,16984,Radioactive Long Island Iced Tea,None,Rum,Vodka,Tequila,Gin,Triple sec,Chambord raspberry liqueur,Midori melon liqueur,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
616,14029,57 Chevy with a White License Plate,None,Creme de Cacao,Vodka,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
### ABV TABLE
# URL of page to be scraped
abv_url = 'http://www.alcoholcontents.com/liquor/'
abv_table = pd.read_html(abv_url)
abv_df = abv_table[0]
abv_df_clean = abv_df.drop(['Cal'], axis=1)
#Rename columns
renamed_df = abv_df_clean.rename(columns={"Liquor / Cocktail / Liqueur":"Spirit", "% Alc (ABV)":"ABV"})
renamed_df.head()

,Spirit,ABV
0,Advocaat,17
1,Aftershock,30
2,Aftershock Cinnamon Schnapps,40
3,Aftershock Cool Citrus Mint Schnapps,40
4,Aguardiente,30


In [10]:
common_spirits = [{"Spirit": "Baijiu", "ABV": 47},
                {"Spirit": "Bourbon", "ABV": 65},
                {"Spirit": "Brandy", "ABV": 42},
                {"Spirit": "Champagne", "ABV": 12},
                {"Spirit": "Cognac", "ABV": 40},                  
                {"Spirit": "Everclear", "ABV": 85},
                {"Spirit": "Gin", "ABV": 37},
                {"Spirit": "Grappa", "ABV": 47},
                {"Spirit": "Rum", "ABV": 47},
                {"Spirit": "Sake", "ABV": 15},
                {"Spirit": "Tequila", "ABV": 45},
                {"Spirit": "Vermouth", "ABV": 19},
                {"Spirit": "Vodka", "ABV": 42},
                {"Spirit": "Whiskey", "ABV": 46},
                {"Spirit": "German Schnapps", "ABV": 30}]

spirits_df = pd.DataFrame(common_spirits)

In [11]:
# Merge spirits_df and abv_df
merge_df = pd.merge(renamed_df, spirits_df, on=["Spirit", "ABV"], how="outer")

In [12]:
abv_sort_df = merge_df.sort_values("Spirit")
abv_sort_df.head(20)

,Spirit,ABV
0,Advocaat,17
1,Aftershock,30
2,Aftershock Cinnamon Schnapps,40
3,Aftershock Cool Citrus Mint Schnapps,40
4,Aguardiente,30
5,Amadeus,5
6,Amaretto,28
7,Amaretto Di Saronno,28
8,Amaro,17
9,Amarula,17


In [13]:
# accented_text = abv_sort_df["Spirit"][19]

In [14]:
import unicodedata

def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text)


# s = strip_accents(accented_text)

# print(s)

In [15]:
result["Drink_ABV"] = 0
result

,idDrink,strDrink,strImageSource,strIngredient1,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,...,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,strMeasure11,strMeasure12,Drink_ABV
0,15997,GG,None,Galliano,Ginger ale,Ice,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,17222,A1,None,Gin,Grand Marnier,Lemon Juice,Grenadine,None,None,None,...,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,13501,ABC,None,Amaretto,Baileys irish cream,Cognac,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,17203,Kir,None,Creme de Cassis,Champagne,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,14229,747,None,Kahlua,Baileys irish cream,Frangelico,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,16100,Amaretto Stone Sour Alternative,None,Sour mix,Amaretto,Tequila,Orange juice,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
614,12658,Banana Strawberry Shake Daiquiri,None,Strawberries,Banana,Apple juice,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
615,16984,Radioactive Long Island Iced Tea,None,Rum,Vodka,Tequila,Gin,Triple sec,Chambord raspberry liqueur,Midori melon liqueur,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0
616,14029,57 Chevy with a White License Plate,None,Creme de Cacao,Vodka,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [16]:
### ABV CALC
for drink in result.iterrows():
    # index of drink is located at drink[0]
    # row of drink data is located at drink[1]
    total_abv = 0
    total_measure = 0
    for ing in range(3,14):
        for liquor in abv_sort_df["Spirit"]:
            if drink[1][ing] and (strip_accents(liquor.lower()) in drink[1][ing].lower()):
        #if len(abv_sort_df.loc[abv_sort_df["Liquor"] == drink[1][ing]]) > 0:
            #abv = abv_sort_df.loc[abv_sort_df["Liquor"] == drink[1][ing]]["ABV"]
                abv = abv_sort_df.loc[abv_sort_df["Spirit"] == liquor]["ABV"]
                abv = abv.values[0]
                amount =  drink[1][ing + 12]
                total_abv += abv * amount

    final_calculation = 0
    for meas in range(15,27):
        if drink[1][meas]:
            total_measure += drink[1][meas]
            final_calculation = total_abv/total_measure
            
    #before for loop add column to result df defaulting to 0 for final ABV
    result.at[drink[0],"Drink_ABV"] = final_calculation

In [17]:
result

,idDrink,strDrink,strImageSource,strIngredient1,strIngredient2,strIngredient3,strIngredient4,strIngredient5,strIngredient6,strIngredient7,...,strMeasure4,strMeasure5,strMeasure6,strMeasure7,strMeasure8,strMeasure9,strMeasure10,strMeasure11,strMeasure12,Drink_ABV
0,15997,GG,None,Galliano,Ginger ale,Ice,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35
1,17222,A1,None,Gin,Grand Marnier,Lemon Juice,Grenadine,None,None,None,...,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4
2,13501,ABC,None,Amaretto,Baileys irish cream,Cognac,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,17203,Kir,None,Creme de Cassis,Champagne,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13
4,14229,747,None,Kahlua,Baileys irish cream,Frangelico,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,16100,Amaretto Stone Sour Alternative,None,Sour mix,Amaretto,Tequila,Orange juice,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24
614,12658,Banana Strawberry Shake Daiquiri,None,Strawberries,Banana,Apple juice,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
615,16984,Radioactive Long Island Iced Tea,None,Rum,Vodka,Tequila,Gin,Triple sec,Chambord raspberry liqueur,Midori melon liqueur,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,41
616,14029,57 Chevy with a White License Plate,None,Creme de Cacao,Vodka,None,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33


In [18]:
# username = postgres
# password = password
# rds_connection_string = f"{username}:{password}@localhost:5432/project_db"
# engine = create_engine(f'postgresql://{rds_connection_string}')

In [19]:
# result.to_sql(name='cocktail_recipes', con=engine, if_exists='replace', index=False)

In [22]:
result.to_csv("cocktail_data_with_img.csv")
# result.to_excel("cocktail_data_with_img.xlsx")